In [1]:
import pandas as pd
import numpy as np

In [2]:
train_dataset = pd.read_csv('../yemoonsaBigdata/datasets/Part2/penguin_X_train.csv')
train_label = pd.read_csv('../yemoonsaBigdata/datasets/Part2/penguin_y_train.csv')

test_dataset = pd.read_csv('../yemoonsaBigdata/datasets/Part2/penguin_X_test.csv')

In [3]:
train_dataset.head(3)

,species,island,sex,bill_length_mm,bill_depth_mm,flipper_length_mm
0,Adelie,Torgersen,NaN,42.0,20.2,190.0
1,Gentoo,Biscoe,FEMALE,43.5,15.2,213.0
2,Adelie,Torgersen,MALE,42.8,18.5,195.0


In [4]:
train_label.head(3)

,body_mass_g
0,4250.0
1,4650.0
2,4250.0


In [5]:
test_dataset.head(3)

,species,island,sex,bill_length_mm,bill_depth_mm,flipper_length_mm
0,Adelie,Torgersen,MALE,42.1,19.1,195.0
1,Gentoo,Biscoe,MALE,45.5,15.0,220.0
2,Adelie,Biscoe,MALE,40.6,18.8,193.0


<br><br>

### **데이터 전처리**

데이터 세트를 불러온 후 데이터에 대한 기본 정보를 확인해보자.

먼저 데이터의 shape은 (240, 6)으로 행은 240개, 열은 6개이다.

6개 변수를 독립 변수로 두고 분석을 진행하도록 한다.

In [6]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            240 non-null    object 
 1   island             240 non-null    object 
 2   sex                232 non-null    object 
 3   bill_length_mm     238 non-null    float64
 4   bill_depth_mm      238 non-null    float64
 5   flipper_length_mm  238 non-null    float64
dtypes: float64(3), object(3)
memory usage: 11.4+ KB


<br>

불러온 데이터프레임은 `info` 함수를 통해 데이터의 column 정보와 결측치가 아닌 정상적인 데이터의 개수, 그리고 데이터 타입을 확인할 수 있다.

출력된 정보를 통해 dataset에 6개의 column이 있으며 sex에 8개, bill_length_mm과 bill_depth_mm, flipper_length_mm에 각각 2개의 결측치가 있음을 확인하였다.

species, island, sex column은 object 변수이며, 나머지 column은 모두 float64이다.

<br>

In [7]:
train = pd.concat([train_dataset, train_label], axis=1)
train.isnull().sum()

species              0
island               0
sex                  8
bill_length_mm       2
bill_depth_mm        2
flipper_length_mm    2
body_mass_g          2
dtype: int64

In [8]:
train.loc[train['sex'].isna() | train['bill_length_mm'].isna() | train['bill_depth_mm'].isna() | train['flipper_length_mm'].isna() | train['body_mass_g'].isna(), :]

,species,island,sex,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,Adelie,Torgersen,NaN,42.0,20.2,190.0,4250.0
6,Gentoo,Biscoe,NaN,44.5,14.3,216.0,4100.0
43,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
66,Adelie,Torgersen,NaN,37.8,17.3,180.0,3700.0
88,Gentoo,Biscoe,NaN,47.3,13.8,216.0,4725.0
89,Adelie,Torgersen,NaN,37.8,17.1,186.0,3300.0
110,Gentoo,Biscoe,NaN,44.5,15.7,217.0,4875.0
229,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN


<br>

결측치가 각기 다른 행에서 발생하지 않아 8개의 행만 처리해주면 된다.

여기서는 간단히 `dropna` 함수를 통해 결측치를 제거하는 방식을 택했다.

여기서 제거된 행의 인덱스 번호가 비어 있는 상태로 데이터가 유지된다.

reset_index를 통해 인덱스를 초기화하여 추후 인덱스로 인한 혼란을 막도록 한다.

<br>

In [9]:
train = train.dropna()
train.reset_index(drop=True, inplace=True)

In [10]:
train.isnull().sum()

species              0
island               0
sex                  0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
dtype: int64

In [11]:
label = ['body_mass_g']
columns = [col for col in train.columns if col not in label]

train_dataset = train[columns]
train_label = train[label]

<br>

결측치 처리를 완료했으므로 붙였던 학습용 데이터를 다시 독립변수와 종속변수로 분리시키도록 하자.

<br>

In [12]:
train_dataset.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm
count,232.000000,232.000000,232.000000
mean,43.990948,17.226293,200.681034
std,5.509760,1.964677,14.064231
min,32.100000,13.200000,172.000000
25%,39.200000,15.700000,190.000000
50%,44.950000,17.350000,197.000000
75%,48.775000,18.725000,212.250000
max,58.000000,21.500000,231.000000


<br>

마찬가지로 `describe` 함수를 통해 평균, 표준편차, 사분위 수 등 기본적인 요약 통계량 값을 확인한다.

분석에 큰 영향을 끼칠만한 이상치가 없는 것으로 보이며, 각 수치형 변수의 변수의 범위가 상이하기 때문에 추후 Scaling이 필요하다는 것을 기억하도록 하자.

<br>

In [13]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            232 non-null    object 
 1   island             232 non-null    object 
 2   sex                232 non-null    object 
 3   bill_length_mm     232 non-null    float64
 4   bill_depth_mm      232 non-null    float64
 5   flipper_length_mm  232 non-null    float64
dtypes: float64(3), object(3)
memory usage: 11.0+ KB


<br>

총 6개의 독립변수 중, 3개의 수치형 변수와 3개의 범주형 변수가 존재한다.

- 여기서 수치형 변수는 모델링을 진행하면서 Scaling을 진행해줄 예정이며,

- 범주형 변수에 대해서는 label encoding이 필요하다.

<br>

Scaling은 평균과 분산에 영향을 줄 수 있기 때문에 학습 데이터로만 한정하여 진행해야 하지만,

label encoding의 경우에는 그렇지 않으므로 현재 단계에서 모든 데이터를 포함하여 label encoding을 진행하도록 하자.

<br>



In [14]:
columns_num = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm']
columns_cat = ['species' ,'island', 'sex']
columns_y = ['body_mass_g']

In [15]:
from sklearn.preprocessing import OneHotEncoder

concat = pd.concat([train_dataset, test_dataset], axis=0)

one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
one_hot_encoder.fit(concat[columns_cat])

train_encoded = one_hot_encoder.transform(train_dataset[columns_cat])
test_encoded = one_hot_encoder.transform(test_dataset[columns_cat])

In [17]:
print(train_encoded)

  (0, 2)	1.0
  (0, 3)	1.0
  (0, 6)	1.0
  (1, 0)	1.0
  (1, 5)	1.0
  (1, 7)	1.0
  (2, 1)	1.0
  (2, 4)	1.0
  (2, 7)	1.0
  (3, 2)	1.0
  (3, 3)	1.0
  (3, 7)	1.0
  (4, 0)	1.0
  (4, 4)	1.0
  (4, 6)	1.0
  (5, 2)	1.0
  (5, 3)	1.0
  (5, 7)	1.0
  (6, 2)	1.0
  (6, 3)	1.0
  (6, 6)	1.0
  (7, 0)	1.0
  (7, 5)	1.0
  (7, 6)	1.0
  (8, 2)	1.0
  :	:
  (223, 6)	1.0
  (224, 2)	1.0
  (224, 3)	1.0
  (224, 7)	1.0
  (225, 0)	1.0
  (225, 4)	1.0
  (225, 7)	1.0
  (226, 1)	1.0
  (226, 4)	1.0
  (226, 7)	1.0
  (227, 1)	1.0
  (227, 4)	1.0
  (227, 6)	1.0
  (228, 2)	1.0
  (228, 3)	1.0
  (228, 7)	1.0
  (229, 0)	1.0
  (229, 5)	1.0
  (229, 6)	1.0
  (230, 1)	1.0
  (230, 4)	1.0
  (230, 6)	1.0
  (231, 2)	1.0
  (231, 3)	1.0
  (231, 6)	1.0


<br>

`OneHotEncoder` 함수를 사용해 label encoding을 진행하자.

sklearn의 preprocessing 모듈에서 `OneHotEncoder` 함수를 import할 수 있다.

`handle_unknown` 파라미터는 fit을 한 뒤 transform을 진행할 때 처음 보는 범주가 등장했을 때 어떻게 처리할 것인지 결정하는 옵션이다.

이 부분에 대해서는 처음 보는 범주가 등장하지 않을 것이라 하고 `ignore` 처리를 해준다.

<br>

In [20]:
train_encoded = pd.DataFrame(train_encoded.todense(), columns=one_hot_encoder.get_feature_names_out())
test_encoded = pd.DataFrame(test_encoded.todense(), columns=one_hot_encoder.get_feature_names_out())

train_dataset = pd.concat([train_dataset[columns_num], train_encoded], axis=1)
test_dataset = pd.concat([test_dataset[columns_num], test_encoded], axis=1)

In [21]:
train_dataset.head(3)

,bill_length_mm,bill_depth_mm,flipper_length_mm,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_FEMALE,sex_MALE
0,43.5,15.2,213.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,42.8,18.5,195.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,53.5,19.9,205.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [24]:
test_dataset.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_FEMALE,sex_MALE
0,42.1,19.1,195.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,45.5,15.0,220.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,40.6,18.8,193.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,39.5,17.8,188.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,45.1,14.5,207.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


<br>

transform 된 데이터는 numpy array로 만들어진 데이터이며,

기존에 사용하던 데이터프레임이 아닌 특정 행렬 좌표에 1.0 값이 존재한다는 방식으로 표현한 희소행렬을 효율적으로 표현한 데이터이다.

이것을 기존 데이터에 적용하기 위해서는 위와 같이 데이터프레임으로 변경시키는 과정이 필요하다.

<br>

### **데이터 모형 구축**



회귀 모형을 만들기 위해 학습용 데이터와 검증용 데이터 분할을 진행한다.

주어진 학습 데이터를 sklearn의 `train_test_split` 함수를 사용하여 학습 데이터와 검증 데이터로 다시 분류할 수 있다.

<br>

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(train_dataset, train_label, test_size=0.3)

<br>

학습 데이터와 검증 데이터가 split 되었으니 수치형 변수에 대해서 Scaling을 진행한다.

sklearn 패키지의 preprocessing 모듈에서 `MinMaxScaler` 함수를 import 하여 Scaling을 진행한다.

<br>

In [25]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train[columns_num])

X_train[columns_num] = scaler.transform(X_train[columns_num])
X_validation[columns_num] = scaler.transform(X_validation[columns_num])
test_dataset[columns_num] = scaler.transform(test_dataset[columns_num])

<br>

`fit` 함수를 사용할 때는 train 시 사용할 데이터만 쓰도록 하자.

그래야만 **평균과 분산에 영향을 미치는 데이터 누수가 발생하지 않는다.**

Scaling까지 완료했으니 선형 회귀 모형을 택하여 회귀분석을 학습하도록 하자.

<br>

In [27]:
from sklearn.linear_model import LinearRegression

model_LR = LinearRegression()
model_LR.fit(X_train, y_train)

y_validation_pred = model_LR.predict(X_validation)
y_validation_pred

array([[3369.07538203],
       [3941.84366567],
       [3684.68129637],
       [3474.04291479],
       [4579.18288721],
       [4129.24099746],
       [4627.76190034],
       [4095.96522753],
       [3538.23902058],
       [4844.77946286],
       [4056.19333446],
       [3123.38681044],
       [4030.11523592],
       [4354.23627696],
       [3686.97774825],
       [4200.02933761],
       [3933.47722857],
       [5222.31769053],
       [3490.59373667],
       [3570.49894373],
       [3263.29221411],
       [4461.90349613],
       [5162.95085326],
       [4330.71951561],
       [3348.4318965 ],
       [4088.23740172],
       [3678.95671534],
       [4901.16109853],
       [3469.71684663],
       [4791.49254541],
       [3538.92278469],
       [3589.37092886],
       [3743.83710123],
       [3225.45250624],
       [4220.17387072],
       [3393.44056903],
       [3435.09762234],
       [3969.37264574],
       [5544.99951872],
       [4814.16369582],
       [3747.15011963],
       [4779.257

<br>

sklearn 패키지의 `LinearRegression` 함수를 사용하면 간단하게 선형회귀를 학습할 수 있다.

분류모형과 마찬가지로 fit 함수를 통해 학습하고 predict 함수로 학습한 결과를 도출시킬 수 있다.

<br>

<br>

### **데이터 모형 평가**

학습이 완료된 선형회귀 모형을 평가하자.

문제에서 요구하는 평가항목인 `RMSE`는 `MSE`에 제곱근을 씌운 형태이며

`MSE`와 거의 동일한 방식이기 때문에 `mean_squared_error` 함수만 import하여 계산할 수 있다.

<br>

In [30]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_validation, y_validation_pred)
rmse = mse ** 0.5


print(f"mse : {mse:.3f}, rmse: {rmse:.3f}")

mse : 80490.048, rmse: 283.708


<br>

문제에서 요구하는 제출물을 만들도록 하자.

<br>

In [31]:
y_pred = model_LR.predict(test_dataset)
y_pred

array([[4134.66386789],
       [5316.83746501],
       [4126.52106497],
       [3514.00696124],
       [4655.847811  ],
       [4092.0430785 ],
       [3395.3532864 ],
       [3797.10974429],
       [3566.35316051],
       [3591.54573772],
       [3432.84768487],
       [5503.80300112],
       [3679.6987895 ],
       [4075.17728595],
       [3768.89096402],
       [4117.53276563],
       [4440.24819249],
       [3560.05321078],
       [3716.19071579],
       [3374.5815527 ],
       [5305.52273799],
       [4203.20746106],
       [5938.08639781],
       [4675.88070962],
       [5367.87801594],
       [5516.66857549],
       [5525.94724866],
       [4246.86227737],
       [3437.80581224],
       [4095.66357838],
       [4244.84440367],
       [3480.15772096],
       [3564.10517446],
       [4603.11977557],
       [3401.45836286],
       [3511.42046741],
       [3311.84975495],
       [3407.09673644],
       [3889.33761156],
       [4998.49098016],
       [3564.55532919],
       [5621.382

In [35]:
pd.DataFrame({
    'body_mass_g': y_pred.squeeze()
}).to_csv('../yemoonsaBigdata/res/004000000.csv', index=False)